# 단일 검증셋

In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 1. Load data
df = pd.read_csv("train.csv")

# 2. 중복 제거
df = df.drop_duplicates()

# 3. Encoding
df['Extracurricular Activities'] = df['Extracurricular Activities'].map({
    'Yes': 1,
    'No': 0
})

# 4. Feature / Target 분리
X = df.drop("Performance Index", axis=1)
y = df["Performance Index"]

# 5. Train / Validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 후보 모델들
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "MLP": MLPRegressor(max_iter=500)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    results[name] = {
        "MSE": mean_squared_error(y_val, pred),
        "RMSE": np.sqrt(mean_squared_error(y_val, pred)),
        "R2": r2_score(y_val, pred)
    }

print("모델별 성능 비교:")
for name, metrics in results.items():
    print(f"{name}: MSE={metrics['MSE']:.5f}, RMSE={metrics['RMSE']:.7f}, R2={metrics['R2']:.7f}")

best_model_name = max(results, key=lambda name: results[name]["R2"])
print(f"\n가장 좋은 모델은: {best_model_name}")

모델별 성능 비교:
LinearRegression: MSE=4.15133, RMSE=2.0374823, R2=0.9889754
Ridge: MSE=4.15135, RMSE=2.0374856, R2=0.9889754
Lasso: MSE=4.98499, RMSE=2.2327092, R2=0.9867615
DecisionTree: MSE=9.28422, RMSE=3.0470021, R2=0.9753441
RandomForest: MSE=5.50168, RMSE=2.3455664, R2=0.9853893
GradientBoosting: MSE=4.55405, RMSE=2.1340214, R2=0.9879059
SVR: MSE=6.34949, RMSE=2.5198199, R2=0.9831378
KNN: MSE=6.15409, RMSE=2.4807443, R2=0.9836567
MLP: MSE=4.19943, RMSE=2.0492506, R2=0.9888477

가장 좋은 모델은: LinearRegression


단일 검증셋 평가: 데이터 분할 방식에 따라 특정 모델이 우연히 더 잘 맞을 수 있습니다. LinearRegression이 그 검증셋에서는 가장 높은 R²을 보였죠.

# 교차 검증

In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# 데이터 로드
train_df = pd.read_csv("train.csv")
train_df = train_df.drop_duplicates()
train_df['Extracurricular Activities'] = train_df['Extracurricular Activities'].map({
    'Yes': 1,
    'No': 0
})
X = train_df.drop(columns=["Performance Index"])
y = train_df["Performance Index"]

# 후보 모델들
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "MLP": MLPRegressor(max_iter=500)
}

results = {}
for name, model in models.items():
    # 교차검증 (5-fold, R^2 기준)
    scores = cross_val_score(model, X, y, cv=5, scoring="r2")
    results[name] = {
        "mean_R2": scores.mean(),
        "std_R2": scores.std()
    }

print("교차검증 결과:")
for name, metrics in results.items():
    print(f"{name}: 평균 R2={metrics['mean_R2']:.10f}, 표준편차={metrics['std_R2']:.10f}")

best_model_name = max(results, key=lambda name: results[name]["mean_R2"])
print(f"\n가장 좋은 모델은: {best_model_name}")


교차검증 결과:
LinearRegression: 평균 R2=0.9885741354, 표준편차=0.0003281541
Ridge: 평균 R2=0.9885741376, 표준편차=0.0003281852
Lasso: 평균 R2=0.9866697053, 표준편차=0.0004827954
DecisionTree: 평균 R2=0.9741508725, 표준편차=0.0005806524
RandomForest: 평균 R2=0.9850015572, 표준편차=0.0005855513
GradientBoosting: 평균 R2=0.9876300454, 표준편차=0.0004224029
SVR: 평균 R2=0.9828817213, 표준편차=0.0003310369
KNN: 평균 R2=0.9824500548, 표준편차=0.0004358799
MLP: 평균 R2=0.9883483977, 표준편차=0.0004383721

가장 좋은 모델은: Ridge


# 교차검증 + 하이퍼파라미터 튜닝(GridSearchCV)으로 Ridge의 최적 알파 값을 찾아주는 코드

In [10]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import os
os.environ["JOBLIB_TEMP_FOLDER"] = "E:\\temp"

# 데이터 로드
train_df = pd.read_csv("train.csv")
train_df = train_df.drop_duplicates()
train_df['Extracurricular Activities'] = train_df['Extracurricular Activities'].map({
    'Yes': 1,
    'No': 0
})
X = train_df.drop(columns=["Performance Index"])
y = train_df["Performance Index"]

# Ridge 모델 정의
ridge = Ridge()

# 탐색할 alpha 값 범위 설정
param_grid = {
    "alpha": [0.01, 0.1, 1, 10, 100, 1000]
}

# GridSearchCV 설정 (5-fold 교차검증, R² 기준)
grid_search = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1
)

# 학습
grid_search.fit(X, y)

# 결과 출력
print("최적 하이퍼파라미터:", grid_search.best_params_)
print("최적 평균 R²:", grid_search.best_score_)
print("최적 모델:", grid_search.best_estimator_)


최적 하이퍼파라미터: {'alpha': 1}
최적 평균 R²: 0.9885741376473879
최적 모델: Ridge(alpha=1)


In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# 데이터 로드
train_df = pd.read_csv("train.csv")
train_df = train_df.drop_duplicates()
train_df['Extracurricular Activities'] = train_df['Extracurricular Activities'].map({
    'Yes': 1,
    'No': 0
})
X = train_df.drop(columns=["Performance Index"])
y = train_df["Performance Index"]

# Ridge 모델 정의
ridge = Ridge()

# alpha 값을 로그 스케일로 세밀하게 탐색
param_grid = {
    "alpha": np.logspace(-3, 3, 20)  # 0.001 ~ 1000 사이에서 20개 값
}

# GridSearchCV 설정 (5-fold 교차검증, R² 기준)
grid_search = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    cv=5,
    scoring="r2",
    n_jobs=1   # Windows 환경에서 인코딩 문제 피하려면 병렬 끄기
)

# 학습
grid_search.fit(X, y)

# 결과 출력
print("최적 하이퍼파라미터:", grid_search.best_params_)
print("최적 평균 R²:", grid_search.best_score_)
print("최적 모델:", grid_search.best_estimator_)


최적 하이퍼파라미터: {'alpha': np.float64(6.158482110660261)}
최적 평균 R²: 0.9885741418135519
최적 모델: Ridge(alpha=np.float64(6.158482110660261))


# 스케일링

In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 모델들
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# 1. Load & preprocess
df = pd.read_csv("train.csv").drop_duplicates()
df['Extracurricular Activities'] = df['Extracurricular Activities'].map({'Yes': 1, 'No': 0})

X = df.drop("Performance Index", axis=1)
y = df["Performance Index"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Pipeline 기반 모델 정의
models = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),
    "Ridge": Pipeline([
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=6.158482110660261))
    ]),
    "Lasso": Pipeline([
        ("scaler", StandardScaler()),
        ("model", Lasso())
    ]),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "SVR": Pipeline([
        ("scaler", StandardScaler()),
        ("model", SVR())
    ]),
    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("model", KNeighborsRegressor())
    ]),
    "MLP": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MLPRegressor(max_iter=500, random_state=42))
    ])
}

# 3. 평가
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    results[name] = {
        "RMSE": np.sqrt(mean_squared_error(y_val, pred)),
        "R2": r2_score(y_val, pred)
    }

print("단일 검증셋 결과:")
for k, v in results.items():
    print(f"{k}: RMSE={v['RMSE']:.4f}, R2={v['R2']:.5f}")
max_r2 = max(results, key=lambda name: results[name]["R2"])
min_rmse = min(results, key=lambda name: results[name]["RMSE"])
print(f"\n가장 높은 R2: {max_r2}")
print(f"\n가장 낮은 오차: {min_rmse}")

단일 검증셋 결과:
LinearRegression: RMSE=2.0375, R2=0.98898
Ridge: RMSE=2.0382, R2=0.98897
Lasso: RMSE=2.7588, R2=0.97979
DecisionTree: RMSE=3.0881, R2=0.97467
RandomForest: RMSE=2.3392, R2=0.98547
GradientBoosting: RMSE=2.1340, R2=0.98791
SVR: RMSE=2.4640, R2=0.98388
KNN: RMSE=3.0790, R2=0.97482
MLP: RMSE=2.1070, R2=0.98821

가장 높은 R2: LinearRegression

가장 낮은 오차: LinearRegression


In [5]:
from sklearn.model_selection import cross_val_score

print("\n5-Fold 교차검증 결과 (R²):")

cv_results = {}
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring="r2")
    # cv_results[name] = (scores.mean(), scores.std())
    cv_results[name] = {
        "mean_R2": scores.mean(),
        "std_R2": scores.std()
    }
    print(f"{name}: 평균={scores.mean():.5f}, 표준편차={scores.std():.5f}")

best_model_name = max(cv_results, key=lambda name: cv_results[name]['mean_R2'])
print(f"\n가장 좋은 모델은: {best_model_name}")


5-Fold 교차검증 결과 (R²):
LinearRegression: 평균=0.98857, 표준편차=0.00033
Ridge: 평균=0.98857, 표준편차=0.00033
Lasso: 평균=0.98002, 표준편차=0.00046
DecisionTree: 평균=0.97404, 표준편차=0.00070
RandomForest: 평균=0.98499, 표준편차=0.00055
GradientBoosting: 평균=0.98763, 표준편차=0.00042
SVR: 평균=0.98337, 표준편차=0.00025
KNN: 평균=0.97252, 표준편차=0.00057
MLP: 평균=0.98801, 표준편차=0.00028

가장 좋은 모델은: LinearRegression


초기 단일 검증 셋 평가에서는 Linear Regression이 가장 우수한 성능을 보였다.
이후 교차검증 결과에서 Ridge Regression이 매우 근소하게 높은 평균 R²를 기록하여,
정규화 효과에 따른 일반화 성능 개선 가능성을 검토하였다. 이에 따라 Ridge 모델에 대해 하이퍼파라미터 튜닝을 수행하였으며,
alpha ≈ 6.16에서 최적의 교차검증 성능을 확인하였다.

그러나 컬럼 간 값의 범위 차이를 고려하여 표준화(Standard Scaling)를 적용한 후,
단일 검증 셋 평가와 5-Fold 교차검증을 다시 수행한 결과,
Linear Regression과 Ridge Regression은 성능 차이가 사실상 없는 수준으로 수렴하였다.
특히 Linear Regression은 Ridge와 동일한 평균 R²와 유사한 분산을 보이면서도,
추가적인 하이퍼파라미터 튜닝이 필요 없는 가장 단순한 구조를 유지하였다.

이에 따라 본 프로젝트에서는
성능, 안정성, 해석 용이성, 모델 단순성을 종합적으로 고려하여
초기 실험에서 확인된 Linear Regression 모델을 최종 모델로 유지하였다.